In [9]:
import numpy as np
import msicpe.san as san
from plotly import express as px
import scipy.signal as signal
import pandas as pd

# **Partie 2**

## **Fonctions generiques**

In [10]:
signal_dict = np.load('apprentissage/a_1.npz') # signal_dict est un dictionnaire
#contenant les données contenues dans le fichier signal.npz
s = signal_dict['s'] # extraction du signal utile
t = signal_dict['t'] # extraction du vecteur temps associé au signal

px.line(x=t, y=s, labels={"x":"Temps (s)", "y":"Amplitude"},
title="a_1.npz").show()

#help(san.trans_fourier)
nu, S = san.trans_fourier(s, t)

S = S - np.mean(S)

S = np.absolute(S)
S = np.log10(S)
S = S / np.max(S) #normalisation de l'amplitude
print(np.max(S))

px.line(x=nu, y=S, labels={"x":"Temps (HZ)", "y":"Amplitude"},
title="TF a_1.npz").show()

1.0


## **analyse des groupes sonores**

lettre = "a"
EnsembleVect = []
vect_freq_moyen = [0]
vect_ampl_moyen = [0]

courbes = []

for i in range (1,6):
    nom = str(lettre) +"_" +str(i) + ".npz"
    signal_dict = np.load('apprentissage/'+nom) 
    
    s = signal_dict['s'] # extraction du signal utile
    t = signal_dict['t'] # extraction du vecteur temps associé au signal
    
    """px.line(x=t, y=s, labels={"x":"Temps (s)", "y":"Amplitude"},
    title=nom).show()"""

    nu, S = san.trans_fourier(s-np.mean(s), t)
    S = np.absolute(S)
    S = S / np.max(S)  #normalisation de l'amplitude
    fondamentale = (san.detect_fondamentale(S, nu))
    S = np.log10(S) +2
    courbes.append(S)

    
    freqs_harmoniques = []
    for l in range (6):
        freqs_harmoniques.append((l+1)*fondamentale)
    #A partir d'ici un fait directement la moyenne des fondamentales et harmoniques
    vect_freq_moyen[0] += (1/5)*fondamentale #terminer la liste de toutes les freqs moyennes et amplitudes
    
    print('san detect pics',san.detect_pics(S,freqs_harmoniques,nu))
    valtest = san.detect_pics(S,freqs_harmoniques,nu) #tuple (freq pic, amplitude pic)

    harmoniques_reelles = valtest[0] #freqeunces reeles des harmoniques
    amplitudes_harmoniques = valtest[1] #amplitudes des harmoniques
    
    print("harmoniques",harmoniques_reelles)

    print("liste freqences pics",vect_freq_moyen)
    print("liste amplitudes pics",vect_ampl_moyen)

df_signaux = pd.DataFrame({
"Temps (Hz)": np.concatenate([nu, nu, nu, nu, nu]),
"Valeur": np.concatenate([courbes[0], courbes[1], courbes[2], courbes[3], courbes[4]]),
"Signal": ["S1"]*len(nu) + ["S2"]*len(nu) + ["S3"]*len(nu) + ["S4"]*len(nu) + ["S5"]*len(nu)
})
px.line(df_signaux, x="Temps (Hz)", y="Valeur", color="Signal"
,title="superposition des courbes").show()

## Procédure des boucles <br>
¨¨¨¨
**<h3>boucle lecture de** *lettres* <br>
¨¨¨¨**boucle lecture des courbes** *1 à 5* <br>
¨¨¨¨¨¨¨¨*san.detect_fondamentale* <br>
¨¨¨¨¨¨¨¨**boucle lecture des harmoniques et amplitudes** <br>
¨¨¨¨¨¨¨¨¨¨¨¨*san.detect_pics* <br>
¨¨¨¨¨¨¨¨¨¨¨¨*Moyenne* <br>
¨¨¨¨¨¨¨¨*Vecteurs de fréquences et amplitudes* <br>
¨¨¨¨¨¨¨¨*Ajout de la tranformée dans le tracé* <br>
¨¨¨¨**boucle parcour des harmoniques** *1 à 5*<br>
¨¨¨¨¨¨¨¨**boucle parcour des vecteurs** <br>
¨¨¨¨¨¨¨¨¨¨¨¨*calcul de la somme des distances pour Ecart-Type* <br>
¨¨¨¨¨¨¨¨*calcul Ecarts types pour chaque harmonique* <br>
</h3>

In [17]:
LETTRES = ["a","e","i","o","u"]
DATA_vect_Freq = {"a":[0,0,0,0,0],"e":[0,0,0,0,0],"i":[0,0,0,0,0],"o":[0,0,0,0,0],"u":[0,0,0,0,0]} #contient le vecteur frequence associé à chaque spectre 1 à 5
DATA_vect_Amp = {"a":[0,0,0,0,0],"e":[0,0,0,0,0],"i":[0,0,0,0,0],"o":[0,0,0,0,0],"u":[0,0,0,0,0]} #contient le vecteur amplitude associé à chaque spectre 1 à 5
DATA_moyennes = {"a":[0,0],"e":[0,0],"i":[0,0],"o":[0,0],"u":[0,0]} ## [frequences moyennes, amplitudes moyennes]
DATA_ecartTypes = {"a":[0,0],"e":[0,0],"i":[0,0],"o":[0,0],"u":[0,0]} ## [écart type frequences, écart type amplitude]


for var_lettre in LETTRES:
    courbes = [] #tranformee de Fourier complète des 5 signaux d'un son
    for var_indice in range (1,6): #varie de 1 à 5
        nom = var_lettre +"_" +str(var_indice) + ".npz" 
        signal_dict = np.load('apprentissage/'+nom) #ouvrir le fichier signal
        
        s = signal_dict['s'] # extraction du signal temporel
        t = signal_dict['t'] # extraction du vecteur temps associé au signal

        nu, S = san.trans_fourier(s, t) #transformee de Fourier
        S = S - np.mean(S)
        S = np.absolute(S)
        fondamentale = (san.detect_fondamentale(S, nu, 0.7*np.max(S))) #on relève la fondamentale tant que ça marche
        S = np.log10(S) #utilisation de l'echelle logarithmique
        S = S / np.max(S)  #normalisation de l'amplitude
        courbes.append(S)

        """A FAIRE : ajouter un moyen de determine le nb d harmoniques a prendre et de creer la longeur de liste automatiquement"""

        temp_freqs_harmoniques = [fondamentale]
        for var_rang_harmonique in range (6): #ajout des 6 harmoniques suivantes
            temp_freqs_harmoniques.append((var_rang_harmonique+1)*fondamentale)
        #A partir d'ici on fait directement la moyenne des fondamentales et harmoniques
        
        SIGNAL_Harmonique_Freq, SIGNAL_Harmonique_Amp= san.detect_pics(S,temp_freqs_harmoniques,nu) #tuple (freq pic, amplitude pic)
        
        DATA_vect_Freq[var_lettre][var_indice-1] = SIGNAL_Harmonique_Freq
        DATA_vect_Amp[var_lettre][var_indice-1] = SIGNAL_Harmonique_Amp
        
        DATA_moyennes[var_lettre][0] += 1/5*SIGNAL_Harmonique_Freq
        DATA_moyennes[var_lettre][1] += 1/5*SIGNAL_Harmonique_Amp
    
    for var_rang_harmonique2 in range(int(len(DATA_vect_Freq[var_lettre][0]))-1):#boucle des harmoniques
        print(int(len(DATA_vect_Freq[var_lettre][0])))
        temp_sommeMesuresFreq, temp_sommeMesuresAmp= 0, 0
        temp_ecartType_Freq, temp_ecartType_Amp = 0, 0
        temp_vect_harmoniques_Freq, temp_vect_harmoniques_Amp = [[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]], [[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]
        
        for var_indice2 in range (5): #boucle des signaux
            temp_sommeMesuresFreq += (DATA_vect_Freq[var_lettre][var_indice2][var_rang_harmonique2] - DATA_moyennes[var_lettre][0][var_rang_harmonique2])**2
            temp_sommeMesuresAmp += (DATA_vect_Amp[var_lettre][var_indice2][var_rang_harmonique2] - DATA_moyennes[var_lettre][1][var_rang_harmonique2])**2
        temp_ecartType_Freq = np.square((1/5)*temp_sommeMesuresFreq)
        temp_ecartType_Amp = np.square((1/5)*temp_sommeMesuresAmp)
        print("signal ",var_indice2, ". harmonique ", var_rang_harmonique2)
        temp_vect_harmoniques_Freq[var_indice2][var_rang_harmonique2] = temp_ecartType_Freq
        temp_vect_harmoniques_Amp[var_indice2][var_rang_harmonique2] = temp_ecartType_Amp
    DATA_ecartTypes[var_lettre][0] = temp_vect_harmoniques_Freq
    DATA_ecartTypes[var_lettre][1] = temp_vect_harmoniques_Amp

    """Tracés des courbes"""
    COURBE_superposition = px.line(x=DATA_moyennes[var_lettre][0], y=DATA_moyennes[var_lettre][1], labels={"x":"", "y":"Amplitude"},
        title=var_lettre, markers='markers', range_x=[0, 5000], range_y=[-1.15,1.15]).show() #trace du vecteur moyen
    
    df_signaux = pd.DataFrame({
    "Temps (Hz)": np.concatenate([nu, nu, nu, nu, nu]),
    "Valeur": np.concatenate([courbes[0], courbes[1], courbes[2], courbes[3], courbes[4]]),
    "Signal": ["S1"]*len(nu) + ["S2"]*len(nu) + ["S3"]*len(nu) + ["S4"]*len(nu) + ["S5"]*len(nu)
    })
    px.line(df_signaux, x="Temps (Hz)", y="Valeur", color="Signal"
    ,title=("superposition des courbes "+var_lettre)).show() #trace des signaux superposes

    


7
signal  4 . harmonique  0
7
signal  4 . harmonique  1
7
signal  4 . harmonique  2
7
signal  4 . harmonique  3
7
signal  4 . harmonique  4
7
signal  4 . harmonique  5


IndexError: list assignment index out of range

## *à faire*

Mesurer l'écart type entre les différentes fréquences et amplitudes lors des test **Ecart Type**
et ainsi comparer l'écart de la valeur mesurée avec la moyenne des signaux testés **Ecart de Mesure**
afin de déterminer si l'**Ecart de Mesure** est inférieur à l'**Ecart Type**

## **Fonction de comparaison du signal avec le dictionnaire**

In [ ]:
'''
A 5 harmoniques
E 3 harmoniques
I 4 harmoniques
O 3 harmoniques
U 5 harmoniques
'''



'\nA 5 harmoniques\nE 3 harmoniques\nI 4 harmoniques\nO 3 harmoniques\nU 5 harmoniques\n'